In [62]:
import os
import pandas as pd

txtFilesDir = "pages/"
txtFiles = os.listdir(txtFilesDir)

files = []

for file in txtFiles:
    if ".txt" in file:
        files.append(txtFilesDir + file)

In [63]:
def extract_file_details(file):
    file_components = file.split(txtFilesDir)[1].split('.txt')[0].split('-')
    
    return {
        'discipline': file_components[0],
        'year': int(file_components[1]),
        'term': file_components[2],
    }

In [72]:
from parser import TermSubjectParser

# Dummy dataframe to dump the courses into
courses = pd.DataFrame()

# Pull course data from each file, adding it to the courses dataframe
for file_index in range(0, len(files)):
    file_details = extract_file_details(files[file_index])
    
    parser = TermSubjectParser(**file_details)
    
    file_courses = parser.get_courses()

    file_courses['discipline'] = file_details['discipline']
    file_courses['calendar_year'] = file_details['year']
    file_courses['term'] = file_details['term']

    if (file_index == 0):
        courses = file_courses
    else:
        courses = pd.concat([courses, file_courses])

# Re-index the courses dataframe
courses = courses.reset_index().set_index(['discipline', 'calendar_year', 'term', 'course', 'section'])

0 courses removed due to badly formed data.
0 courses removed due to badly formed data.
0 courses removed due to badly formed data.
0 courses removed due to badly formed data.
0 courses removed due to badly formed data.
0 courses removed due to badly formed data.


In [177]:
courses_of_interest = pd.read_csv('./data/courses-of-interest.tsv', sep='\t', header=0)

sections_of_interest = pd.merge(courses_of_interest.set_index(['discipline','calendar_year','term', 'course']), courses.reset_index(4), left_index=True, right_index=True)
sections_of_interest = sections_of_interest.set_index('section', append=True)

sections_of_interest

admin.duration  admin.isOpen  \
discipline calendar_year term   course   section                                
HIS        2019          fall   HIS 1110 A00          FullSess.          True   
                                         A00          FullSess.          True   
                                HIS 3120 A00          FullSess.          True   
                                HIS 3124 A00          FullSess.          True   
                                         A00          FullSess.          True   
                                HIS 3150 A00          FullSess.          True   
                                         A00          FullSess.          True   
                                HIS 4100 A00          FullSess.          True   
                                         B00          FullSess.          True   
                                HIS 4192 A00          FullSess.          True   
                                HIS 4380 A00          FullSess.          True   
                                HIS 4397 A00          FullSess.          True   
           2020          winter HIS 3125 A00          FullSess.          True   
                                         A00          FullSess.          True   
                                HIS 3190 A00          FullSess.          True   
                                         A00          FullSess.          True   
                                HIS 4135 D00          FullSess.          True   
                                HIS 4360 A00          FullSess.          True   
                                HIS 4364 A00          FullSess.          True   
                                         A00          FullSess.          True   

                                                 admin.type  \
discipline calendar_year term   course   section              
HIS        2019          fall   HIS 1110 A00            LEC   
                                         A00            LEC   
                                HIS 3120 A00            LEC   
                                HIS 3124 A00            LEC   
                                         A00            LEC   
                                HIS 3150 A00            LEC   
                                         A00            LEC   
                                HIS 4100 A00            SEM   
                                         B00            SEM   
                                HIS 4192 A00            SEM   
                                HIS 4380 A00            SEM   
                                HIS 4397 A00            SEM   
           2020          winter HIS 3125 A00            LEC   
                                         A00            LEC   
                                HIS 3190 A00            LEC   
                                         A00            LEC   
                                HIS 4135 D00            SEM   
                                HIS 4360 A00            LEC   
                                HIS 4364 A00            LEC   
                                         A00            LEC   

                                                               prof weekday  \
discipline calendar_year term   course   section                              
HIS        2019          fall   HIS 1110 A00            Eric Allina      Mo   
                                         A00      Meredith Terretta      Th   
                                HIS 3120 A00                  Staff      Th   
                                HIS 3124 A00        Richard Connors      Mo   
                                         A00        Richard Connors      We   
                                HIS 3150 A00         Heather Murray      Tu   
                                         A00         Heather Murray      Th   
                                HIS 4100 A00            Eric Allina      Tu   
                                         B00         Heather Murray      Th   
          

In [185]:
sections_of_interest = sections_of_interest.reset_index()[[
    'calendar_year',
    'term',
    'course',
    'section',
    'prof',
    'weekday',
    'time.start',
    'time.end',
    'title',
    'description',
    'extraDetails'
]].set_index(['calendar_year', 'term', 'course', 'section'])

sections_of_interest.to_csv('data/courses-of-interest-detailed.csv')

sections_of_interest

prof weekday time.start  \
calendar_year term   course   section                                         
2019          fall   HIS 1110 A00            Eric Allina      Mo      11:30   
                              A00      Meredith Terretta      Th      13:00   
                     HIS 3120 A00                  Staff      Th      08:30   
                     HIS 3124 A00        Richard Connors      Mo      10:00   
                              A00        Richard Connors      We      08:30   
                     HIS 3150 A00         Heather Murray      Tu      13:00   
                              A00         Heather Murray      Th      11:30   
                     HIS 4100 A00            Eric Allina      Tu      08:30   
                              B00         Heather Murray      Th      14:30   
                     HIS 4192 A00                  Staff      We      08:30   
                     HIS 4380 A00       M. Mark Stolarik      Th      19:00   
                     HIS 4397 A00        Richard Connors      We      14:30   
2020          winter HIS 3125 A00        Richard Connors      Mo      13:00   
                              A00        Richard Connors      We      11:30   
                     HIS 3190 A00        Ryme Seferdjeli      Mo      11:30   
                              A00        Ryme Seferdjeli      Th      13:00   
                     HIS 4135 D00                  Staff      Th      08:30   
                     HIS 4360 A00                  Staff      We      19:00   
                     HIS 4364 A00                  Staff      Mo      16:00   
                              A00                  Staff      We      14:30   

                                      time.end  \
calendar_year term   course   section            
2019          fall   HIS 1110 A00        12:50   
                              A00        14:20   
                     HIS 3120 A00        11:20   
                     HIS 3124 A00        11:20   
                              A00        09:50   
                     HIS 3150 A00        14:20   
                              A00        12:50   
                     HIS 4100 A00        11:20   
                              B00        17:20   
                     HIS 4192 A00        11:20   
                     HIS 4380 A00        21:50   
                     HIS 4397 A00        17:20   
2020          winter HIS 3125 A00        14:20   
                              A00        12:50   
                     HIS 3190 A00        12:50   
                              A00        14:20   
                     HIS 4135 D00        11:20   
                     HIS 4360 A00        21:50   
                     HIS 4364 A00        17:20   
                              A00        15:50   

                                                                                   title  \
calendar_year term   course   section                                                      
2019          fall   HIS 1110 A00                         Introduction to Global History   
                              A00                         Introduction to Global History   
                     HIS 3120 A00      Selected Topics in the History of Canada's Reg...   
                     HIS 3124 A00                              Britain from 1485 to 1800   
                              A00                              Britain from 1485 to 1800   
                     HIS 3150 A00                    Selected Topics in American History   
                              A00                    Selected Topics in American History   
                     HIS 4100 A00                      Seminar in History Across Borders   
                              B00                      Seminar in History Across Borders   
                     HIS 4192 A00      Seminar in the History of the Middle East and ...   
                     HIS 4380 A00                      Seminar in North American History   
                